In [80]:
import pandas as pd
import numpy as np
from sklearn.pipeline import make_pipeline
from skrebate import ReliefF,SURF,TuRF
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import train_test_split
import matplotlib
import matplotlib.pyplot as plt
plt.rcParams['text.usetex'] = True #Let TeX do the typsetting
plt.rcParams['text.latex.preamble'] = [r'\usepackage{sansmath}', r'\sansmath'] #Force sans-serif math mode (for axes labels)
plt.rcParams['font.family'] = 'sans-serif' # ... for regular text
plt.rcParams['font.sans-serif'] = 'Helvetica, Avant Garde, Computer Modern Sans serif' # Choose a nice font here
from statsmodels.stats.outliers_influence import variance_inflation_factor    

%matplotlib inline

In [74]:
data = pd.read_pickle('ALLDATA.pkl')
data=data.loc[-data['Wait Time (s)'].isin(['Err1','Err2','Err3','Err4','Err5'])]
data=data.loc[data['Age_9-12'].isin([0,1])]


data=data.loc[:,['Wait Time (s)','Speed Limit', 'Lane Width', 'Minimum Gap', 'Mean Arrival Rate', 'AV', 
                   'Full Braking Before Impact_-1.0', 'Full Braking Before Impact_1', 
                   'Full Braking Before Impact_2', 'Full Braking Before Impact_3', 'Snowy',
                   'One way', 'two way', 'Two way with median', 'Night', 'numcars', 
                    'Age_9-12', 'Age_15-18', 'Age_12-15', 'Age_18 - 24', 'Age_25 - 29', 'Age_30 - 39', 'Age_50 - 59', 'Age_60+', 'Gender_Female', 'Occupation_Employed', 'Occupation_Student', 'Occupation_Unemployed', 
                   'Occupation_kid', 'Education_Bachelors degree', 'Education_College/University student', 
                   'Education_Doctorate degree', 'Education_High school diploma', 'Education_Masters degree', 
                   'Education_Professional degree', 'driving license_Yes', 'mode_Bike', 'mode_Car',
                   'mode_Public Transit', 'mode_Walking', 'workwalk_No', 'workwalk_Sometimes', 'workwalk_Yes', 
                   'shopwalk_No', 'shopwalk_Sometimes', 'shopwalk_Yes', 'Vrexp_Yes']]       #numwalk and VRexpnum removed because of some false inputs in the data should be fixed later


data=data.apply(pd.to_numeric, errors='coerce')

def FullAV(row):
    if row['AV']==100:
        return 1
    else:
        return 0
def MixedAV(row):
    if row['AV']>0 and row['AV']<100:
        return 1
    else:
        return 0   
def HDV(row):
    if row['AV']==0:
        return 1
    else:
        return 0   
data=data[data.Occupation_kid==0]                         #remove kids


data['FullAV']=data.apply (lambda row: FullAV(row), axis=1)
data['Mixed']=data.apply (lambda row: MixedAV(row), axis=1)
#data['HDV']=data.apply (lambda row: HDV(row), axis=1)
data['over50']=data['Age_50 - 59']+data['Age_60+']
data=data.drop(columns=['Age_9-12', 'Age_15-18', 'Age_12-15','Occupation_kid','AV','Age_50 - 59','Age_60+'])

In [75]:
data=data.apply(pd.to_numeric, errors='coerce')
def calculate_vif_(X, thresh=20.0):
    variables = list(range(X.shape[1]))
    dropped = True
    while dropped:
        dropped = False
        vif = [variance_inflation_factor(X.iloc[:, variables].values, ix)
               for ix in range(X.iloc[:, variables].shape[1])]

        maxloc = vif.index(max(vif))
        if max(vif) > thresh:
            print('dropping \'' + X.iloc[:, variables].columns[maxloc] +
                  '\' at index: ' + str(maxloc))
            del variables[maxloc]
            dropped = True

    print('Remaining variables:')
    print(X.columns[variables])
    return X.iloc[:, variables]

data=calculate_vif_(data, thresh=180.0)


/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/statsmodels/stats/outliers_influence.py:181: RuntimeWarning: divide by zero encountered in double_scalars
  vif = 1. / (1. - r_squared_i)


dropping 'Full Braking Before Impact_-1.0' at index: 5
dropping 'Full Braking Before Impact_1' at index: 5
dropping 'One way' at index: 8
dropping 'Occupation_Employed' at index: 16
dropping 'Education_Bachelors degree' at index: 18
dropping 'mode_Bike' at index: 24
dropping 'workwalk_No' at index: 27
dropping 'shopwalk_Yes' at index: 31
Remaining variables:
Index(['Wait Time (s)', 'Speed Limit', 'Lane Width', 'Minimum Gap',
       'Mean Arrival Rate', 'Full Braking Before Impact_2',
       'Full Braking Before Impact_3', 'Snowy', 'two way',
       'Two way with median', 'Night', 'numcars', 'Age_18 - 24', 'Age_25 - 29',
       'Age_30 - 39', 'Gender_Female', 'Occupation_Student',
       'Occupation_Unemployed', 'Education_College/University student',
       'Education_Doctorate degree', 'Education_High school diploma',
       'Education_Masters degree', 'Education_Professional degree',
       'driving license_Yes', 'mode_Car', 'mode_Public Transit',
       'mode_Walking', 'workwalk_Som

In [81]:
# Wait time freq table.


size, scale = 1000, 10
waittimes = pd.Series(data['Wait Time (s)'])

waittimes.plot.hist(grid=True, bins=50, rwidth=0.9,
                   color='#607c8e')
#plt.title('Waiting Time frequency distribution for all participants')
plt.xlabel('Wait Time (S)')
plt.ylabel('Counts')
plt.grid(axis='y', alpha=1)
plt.savefig('waitfreq.pdf')

FileNotFoundError: [Errno 2] No such file or directory: 'latex': 'latex'

Error in callback <function install_repl_displayhook.<locals>.post_execute at 0x7f8d5e1da268> (for post_execute):


/home/arash/anaconda3/envs/py3.6/lib/python3.6/site-packages/matplotlib/font_manager.py:1241: UserWarning: findfont: Font family ['sans-serif'] not found. Falling back to DejaVu Sans.
  (prop.get_family(), self.defaultFamily[fontext]))


FileNotFoundError: [Errno 2] No such file or directory: 'latex': 'latex'

FileNotFoundError: [Errno 2] No such file or directory: 'latex': 'latex'

<Figure size 432x288 with 1 Axes>

In [65]:
features5, labels5 = data.drop(columns=['Wait Time (s)'], axis=1).values, data['Wait Time (s)'].values

In [66]:
ReliefWeights5=pd.DataFrame(index=range(features5.shape[1]),columns=['Covariate','Importance'])
fs5 = SURF()
fs5.fit(features5, labels5)
i=0
for feature_name, feature_score in zip(data.drop(columns=['Wait Time (s)'],axis=1).columns,
                                       fs5.feature_importances_):
    ReliefWeights5.iloc[i,0] = feature_name
    ReliefWeights5.iloc[i,1]= feature_score
    i=i+1

In [67]:
ReliefWeights5.to_pickle('Surf.pkl')      #save ReliefAvg

In [68]:
ReliefWeights5.sort_values(['Importance'],ascending=False)

,Covariate,Importance
28,shopwalk_No,0.029762
19,Education_High school diploma,0.0249818
33,over50,0.0219528
23,mode_Car,0.0192792
27,workwalk_Yes,0.0186831
3,Mean Arrival Rate,0.0180707
20,Education_Masters degree,0.0172059
6,Snowy,0.0111135
11,Age_18 - 24,0.0109934
16,Occupation_Unemployed,0.0102918
